In [1]:
import sys
from itertools import count
from torch import autograd
from torch_geometric.utils import dense_to_sparse
import copy

sys.path.append('../')
from models.gcn import *
from utils.datasets import *

In [2]:
import os
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))

CUDA_VISIBLE_DEVICES: 0


In [3]:
import torch

print("CUDA Device Count:", torch.cuda.device_count())

for i in range(torch.cuda.device_count()):
    print(f"Device {i}: {torch.cuda.get_device_name(i)}")

CUDA Device Count: 1
Device 0: NVIDIA H200 MIG 1c.3g.71gb


In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [5]:
from models.trainable import *

In [6]:
from meta_pgd import *

In [7]:
# dataset_directory = "../Cora"
cora_dataset = Planetoid(root='', name='Cora')
data = cora_dataset[0].to(device)
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [8]:
data.train_mask[0:248]=True

In [9]:
data.val_mask[0:248] = False

In [10]:
data.val_mask[248:500] = True

In [11]:
data.test_mask[140+500:]=True

In [12]:
data.test_mask[140+500:].sum()

tensor(2068, device='cuda:0')

In [13]:
data.val_mask[248:140+500].sum()

tensor(392, device='cuda:0')

In [14]:
data.train_mask.sum()

tensor(248, device='cuda:0')

In [15]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [64]).to(device)

In [16]:
model.reset_parameters()
train = Trainable(model)
train.fit(data, 200)

Epoch 0, Train Loss - 1.9450644254684448, Val Loss - 1.7690825462341309, Val Accuracy - 0.49489795918367346
Epoch 20, Train Loss - 0.02871800772845745, Val Loss - 0.68980872631073, Val Accuracy - 0.8010204081632653
Epoch 40, Train Loss - 0.010691264644265175, Val Loss - 0.6796865463256836, Val Accuracy - 0.8061224489795918
Epoch 60, Train Loss - 0.021859560161828995, Val Loss - 0.6291465163230896, Val Accuracy - 0.8010204081632653
Epoch 80, Train Loss - 0.021179623901844025, Val Loss - 0.634373128414154, Val Accuracy - 0.7933673469387755
Epoch 100, Train Loss - 0.0160499420017004, Val Loss - 0.6341055631637573, Val Accuracy - 0.7959183673469388
Epoch 120, Train Loss - 0.014182561077177525, Val Loss - 0.6532086730003357, Val Accuracy - 0.7959183673469388
Epoch 140, Train Loss - 0.01754961535334587, Val Loss - 0.6482744216918945, Val Accuracy - 0.8112244897959183
Epoch 160, Train Loss - 0.012850947678089142, Val Loss - 0.6511685252189636, Val Accuracy - 0.8061224489795918
Epoch 180, Trai

In [17]:
loss, acc = train.test(data)
print(f'Loss: {loss}, Acc: {acc}')

Loss: 0.5219756960868835, Acc: 0.8452611218568665


# MetaPGD Tests

In [18]:
attacker = MetaPGD(data, device=device)

In [19]:
attacker.setup_surrogate(
    model,
    labeled_nodes=data.train_mask,
    unlabeled_nodes=data.test_mask,
    lambda_=0.0,
)

In [20]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [21]:
data.train_mask.sum()

tensor(248, device='cuda:0')

In [22]:
data.test_mask.sum()

tensor(2068, device='cuda:0')

In [23]:
data.val_mask.sum()

tensor(392, device='cuda:0')

In [24]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [25]:
attacker.reset()
#losses,
best_loss, best_pert = attacker.attack(0.05,base_lr=0.01, grad_clip=1, iterations=400)

Running PGD Attack...:   0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor(258, device='cuda:0')
263 tensor

In [26]:
attacker.loss_list

[tensor(0.8438, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.6395, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.5208, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.4211, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.3593, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.3108, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.2738, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.2438, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.2193, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.2000, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.1825, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.1666, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.1541, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.1406, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.1273, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.1150, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.1023, device='cuda:0', grad_fn=<MeanBackward0>

In [33]:
new_data = handle_new_edges(data, attacker, device)

Updated edge index: torch.Size([2, 11072])


In [34]:
loss, acc = train.test(new_data)
print(f'Loss: {loss}, Acc: {acc}')

Loss: 0.8418100476264954, Acc: 0.7412959381044487


In [35]:
loss, acc = train.test(data)
print(f'Loss: {loss}, Acc: {acc}')

Loss: 0.7666090130805969, Acc: 0.7654738878143134


In [36]:
model2 = GCN(data.x.shape[1], cora_dataset.num_classes, [64]).to(device)

In [37]:
model2.reset_parameters()
train = Trainable(model2)
train.fit(new_data, 200)

Epoch 0, Train Loss - 1.9522827863693237, Val Loss - 1.795912504196167, Val Accuracy - 0.3903061224489796
Epoch 20, Train Loss - 0.0434366874396801, Val Loss - 0.8872222304344177, Val Accuracy - 0.7193877551020408
Epoch 40, Train Loss - 0.022290103137493134, Val Loss - 0.936191737651825, Val Accuracy - 0.7321428571428571
Epoch 60, Train Loss - 0.0303592998534441, Val Loss - 0.8773747086524963, Val Accuracy - 0.7295918367346939
Epoch 80, Train Loss - 0.023370638489723206, Val Loss - 0.879871129989624, Val Accuracy - 0.7219387755102041
Epoch 100, Train Loss - 0.01833762228488922, Val Loss - 0.8985812664031982, Val Accuracy - 0.7295918367346939
Epoch 120, Train Loss - 0.017679989337921143, Val Loss - 0.9322783946990967, Val Accuracy - 0.7219387755102041
Epoch 140, Train Loss - 0.014953047968447208, Val Loss - 0.9448068737983704, Val Accuracy - 0.7091836734693877
Epoch 160, Train Loss - 0.015368962660431862, Val Loss - 0.9612876176834106, Val Accuracy - 0.7142857142857143
Epoch 180, Train 

In [38]:
loss, acc = train.test(new_data)
print(f'Loss: {loss}, Acc: {acc}')

Loss: 0.8876636028289795, Acc: 0.7263056092843327


# Meta Greedy Tests

In [ ]:
from greedy_gd import *

In [ ]:
device = 'cuda:0'

In [ ]:
attacker = Metattack(data, device=device)
attacker.setup_surrogate(
    model,
    labeled_nodes=data.train_mask,
    unlabeled_nodes=data.test_mask,
    lambda_=0.0,
)

In [11]:
import torch

# Number of GPUs visible to PyTorch
print("CUDA Device Count:", torch.cuda.device_count())

# List all visible devices
for i in range(torch.cuda.device_count()):
    print(f"Device {i}: {torch.cuda.get_device_name(i)}")

CUDA Device Count: 4
Device 0: NVIDIA H200
Device 1: NVIDIA H200 MIG 1g.18gb
Device 2: NVIDIA H200 MIG 1g.18gb
Device 3: NVIDIA H200 MIG 1g.18gb


In [12]:
attacker.reset()
attacker.attack(1.00)

Peturbing graph...:   0%|          | 0/5278 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
new_data = handle_new_edges(data, attacker, device)

Updated edge index: torch.Size([2, 11078])


In [11]:
loss, acc = train.test(new_data)
print(f'Loss: {loss}, Acc: {acc}')

Loss: 0.6588292121887207, Acc: 0.799


In [12]:
loss, acc = train.test(data)
print(f'Loss: {loss}, Acc: {acc}')

Loss: 0.6605153679847717, Acc: 0.8


In [30]:
model2 = GCN(data.x.shape[1], cora_dataset.num_classes, [64]).to(device)

In [31]:
model2.reset_parameters()
train = Trainable(model2)
train.fit(new_data, 200)

Epoch 0, Train Loss - 1.946224570274353, Val Loss - 1.8570330142974854, Val Accuracy - 0.325
Epoch 20, Train Loss - 0.11124295741319656, Val Loss - 2.7840189933776855, Val Accuracy - 0.408
Epoch 40, Train Loss - 0.04103148356080055, Val Loss - 3.818260908126831, Val Accuracy - 0.412
Epoch 60, Train Loss - 0.03561371564865112, Val Loss - 3.7860445976257324, Val Accuracy - 0.411
Epoch 80, Train Loss - 0.03578553348779678, Val Loss - 3.9414350986480713, Val Accuracy - 0.417
Epoch 100, Train Loss - 0.035183899104595184, Val Loss - 4.1424689292907715, Val Accuracy - 0.407
Epoch 120, Train Loss - 0.030303457751870155, Val Loss - 4.432448387145996, Val Accuracy - 0.401
Epoch 140, Train Loss - 0.03577311709523201, Val Loss - 4.602138519287109, Val Accuracy - 0.399
Epoch 160, Train Loss - 0.021818237379193306, Val Loss - 4.818694591522217, Val Accuracy - 0.405
Epoch 180, Train Loss - 0.021507030352950096, Val Loss - 4.969539165496826, Val Accuracy - 0.405
Epoch 200, Train Loss - 0.0288426186889